In [1]:
# Point source matching script between heritage programme and Swift source catalogue2
# This script creats a source list (cross_match_xmm_swift.csv) of matched object between XMM and Swift 
# observations
# AUTHOR : Samaresh Mondal (smondal@camk.edu.pl)
# DATE   : 2021 Sep 08

In [44]:
import numpy as np
import math
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table

In [45]:
# Count rate to flux conversion using nH=1e22, powelaw index=2.0 and normaization=2.212E-03
# Swift
sw_ctr_to_flux = 5.742E-11  # 0.3-10 keV band flux from 0.3-10 keV band count rate
# XMM
pn_ctr_to_flux = 5.111E-12  # 0.3-10 keV band flux from 0.2-12 keV band count rate
m_ctr_to_flux = 1.461E-11   # 0.3-10 keV band flux from 0.2-12 keV band count rate

In [46]:
# Observation ids of heritage pointings
obsids_heritage=[['1','0886010101'],['2','0886010401'],['3','0886010501'],
                 ['4','0886010601'],['5','0886010301'],['6','0886010701'],
                 ['7','0886010801'],['8','0886010201'],['9','0886010901'],
                 ['10','0886011001'],['11','0886011101'],['12','0886011201'],
                 ['13','0886020301'],['14','0886020101'],['15','0886020301'],
                 ['16','0886011301']]

In [47]:
df=pd.read_csv("2SXPS_Sources.csv",sep=',',header=None,usecols=[0,1,2,3,44,47],skiprows=1,\
               names=['2SXPS_ID','IAUName','RA','Decl','Rate_band0','HR1'])

In [48]:
df.head()

,2SXPS_ID,IAUName,RA,Decl,Rate_band0,HR1
0,1,2SXPS J002517.5+640451,6.32300,64.08109,0.896800,0.570
1,2,2SXPS J002533.4+640903,6.38929,64.15111,0.038400,0.288
2,3,2SXPS J164421.8+573614,251.09112,57.60415,0.000999,-0.149
3,4,2SXPS J002539.8+640905,6.41618,64.15140,0.065960,0.396
4,5,2SXPS J164458.1+573816,251.24230,57.63791,0.000038,-0.159


In [49]:
ra_sw = np.array(df["RA"])
dec_sw = np.array(df["Decl"])
ctr_sw = np.array(df["Rate_band0"])      # Count rate in 0.3-10 keV band
HR_sw = np.array(df["HR1"])

In [53]:
valuesep = 15.0 # Seperation value to consider good match. You may change this value
fp1 = open("cross_match_xmm_swift.csv","w")
fp1.write("   %s        %s      %s    %s   %s %s %s %s %s\n"%("RA","DEC","XMM-ID","RA(sw)",\
                                "DEC(sw)","Sig","XMM-flux(0.3-10keV)","sw-flux(0.3-10keV)","HR"))
for i in range(len(obsids_heritage)):
    #print(i)
    # Reading the xmm source list and matching with chandra source catalogue
    index = i
    obsid = obsids_heritage[index][1]
    xmm = fits.getdata('M1S001M2S002PNS003_'+str(obsid)+'_emllist_formatted.fits', 1)
    txmm = Table(xmm)

    ra_xmm = txmm['RA']                           # in degree
    dec_xmm = txmm['DEC']                         # in degree
    pos_err_xmm = txmm['RADEC_ERR']               # in arcsec
    srcnumber_xmm = txmm['ML_ID_SRC']
    pos_totalerr_xmm = txmm['TOTAL_RADEC_ERR']
 
    # EPIC-PN count rates
    pn_ctr = txmm['RATE_pn']                      # 0.2-12.0 keV
    pn_ctr1 = txmm['RATE_pn_1']                   # 0.2-0.5 keV
    pn_ctr2 = txmm['RATE_pn_2']                   # 0.5-1.0 keV
    pn_ctr3 = txmm['RATE_pn_3']                   # 1.0-2.0 keV
    pn_ctr4 = txmm['RATE_pn_4']                   # 2.0-4.5 keV
    pn_ctr5 = txmm['RATE_pn_5']                   # 4.5-12.0 keV
    pn_ctr6 = pn_ctr1+pn_ctr2+pn_ctr3+pn_ctr4     # 0.2-4.5 keV band
    pn_HR = (pn_ctr3+pn_ctr4+pn_ctr5)/(pn_ctr1+pn_ctr2) # 1-12 keV/0.2-1 keV
    
    pn_ctr_err1 = txmm['RATE_ERR_pn_1']
    pn_ctr_err2 = txmm['RATE_ERR_pn_2']
    pn_ctr_err3 = txmm['RATE_ERR_pn_3']
    pn_ctr_err4 = txmm['RATE_ERR_pn_4']
    pn_ctr_err5 = txmm['RATE_ERR_pn_5']
    pn_ctr_err6 = np.sqrt(pow(pn_ctr_err1,2.0)+pow(pn_ctr_err2,2.0)+pow(pn_ctr_err3,2.0)+pow(pn_ctr_err4,2.0))
    #HR_err = 
    
    # EPIC-PN count rates
    m1_ctr = txmm['RATE_m1']                      # 0.2-12.0 keV
    m1_ctr1 = txmm['RATE_m1_1']                   # 0.2-0.5 keV
    m1_ctr2 = txmm['RATE_m1_2']                   # 0.5-1.0 keV
    m1_ctr3 = txmm['RATE_m1_3']                   # 1.0-2.0 keV
    m1_ctr4 = txmm['RATE_m1_4']                   # 2.0-4.5 keV
    m1_ctr5 = txmm['RATE_m1_5']                   # 4.5-12.0 keV
    m1_ctr6 = pn_ctr1+pn_ctr2+pn_ctr3+pn_ctr4     # 0.2-4.5 keV band
    m1_HR = (pn_ctr3+pn_ctr4+pn_ctr5)/(pn_ctr1+pn_ctr2) # 1-12 keV/0.2-1 keV
    
    
    # EPIC-PN flux 0.5-7 keV band
    pn_flux = (pn_ctr2+pn_ctr3+pn_ctr4)*pn_ctr_to_flux
    
    # PN count rate in full band 0.2-12 keV
    pn_ctr = txmm['RATE_pn']
    pn_ctr_err = txmm['RATE_ERR_pn']
    
    
    # Change the coordinate RA and DEC into a zip
    coo_xmm = SkyCoord(ra_xmm*u.degree, dec_xmm*u.degree)
    coo_sw = SkyCoord(ra_sw*u.degree, dec_sw*u.degree)
    
    # Cross match between XMM and erosita source catalogue
    idx1, d2d1, d3d1 = coo_xmm.match_to_catalog_sky(coo_sw)
    
    # Define max sep in arcsec to consider good match
    max_sep = valuesep * u.arcsec
    # Apply mask true or false according to seperation limit 
    sep_cons1 = d2d1 < max_sep
    
    coo_xmm_matches1 = coo_xmm[sep_cons1]
    
    ra_xmm1 = ra_xmm[sep_cons1]
    dec_xmm1 = dec_xmm[sep_cons1]
    pn_flux1 = pn_ctr[sep_cons1]*pn_ctr_to_flux
    ra_sw1 = ra_sw[idx1[sep_cons1]]
    dec_sw1 = dec_sw[idx1[sep_cons1]]
    sw_flux1 = ctr_sw[idx1[sep_cons1]]*sw_ctr_to_flux
    HR1 = HR[sep_cons1]
    
    for k in range(len(ra_xmm1)):
        fp1.write("%f %f %d %f %f %g %g %g\n"%(ra_xmm1[k],dec_xmm1[k],int(obsid),ra_sw1[k],dec_sw1[k],\
                                               pn_flux1[k],sw_flux1[k],HR1[k]))
        
fp1.close()

/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/export/work/kalypso/python3_lib/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
